In [91]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import datetime

In [30]:
# fileName = ticker + company name, use this to find a particular stock
path = r'E:\NYU MathFin\Courses\project & presentation\Project\Data\allstocks'
fileNames = [ f for f in listdir(path) if isfile(join(path,f)) ]
fileNames = [ f.strip('.csv') for f in fileNames]

In [87]:
# calculate the market capitalization of each stock
#
# df_mktcap = pd.DataFrame()
#
# for fileName in fileNames:
#    df = pd.read_csv(path+'\\'+fileName+'.csv', parse_dates=True, index_col=['date'])
#    df_mktcap_new = pd.DataFrame(data=df['PRC']*df['SHROUT'],columns=[fileName])
#    df_mktcap = pd.concat([df_mktcap,df_mktcap_new],axis=1)
#
# df_mktcap.to_csv(path_or_buf = path+'\\'+'MarketCap.csv')
# this DF can be used to determine for which days the stock has data

In [101]:
df_mktcap = pd.read_csv(path+'\\'+'MarketCap.csv', parse_dates=True, index_col=['date'])


,AAL_ALEXANDER & ALEXANDER SVCS INC,AAL_AMERICAN AIRLINES GROUP INC,AAPL_APPLE INC,AAP_ADVANCE AUTO PARTS INC,AAV_AVATEX CORP,AA_ALCOA CORP,AA_ALCOA INC,ABBV_ABBVIE INC,ABC_AMERICAN BROADCASTING COS INC,ABC_AMERISOURCEBERGEN CORP,...,XYL_XYLEM INC,YNR_YOUNG & RUBICAM INC,YRCW_Y R C WORLDWIDE INC,YUMC_YUM CHINA HOLDINGS INC,YUM_YUM BRANDS INC,ZBH_ZIMMER BIOMET HOLDINGS INC,ZE_ZENITH ELECTRONICS CORP,ZION_ZIONS BANCORPORATION,ZRN_ZURN INDUSTRIES INC,ZTS_ZOETIS INC
date,,,,,,,,,,,,,,,,,,,,,
1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1058034.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107262.250,NaN
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1040632.125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102510.125,NaN
1980-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1026710.625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109298.875,NaN
1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002348.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110656.625,NaN
1980-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1058034.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109977.750,NaN


In [110]:
# create subperiods of time for test
subperiod = [];
for i in range(7):
    pair = (datetime.datetime(1982+i*5,1,1),datetime.datetime(1982+i*5+4,12,31)) 
    subperiod.append(pair)

In [142]:
# if a stock is ready for sampling in that period, put 1.
df_sampling = pd.DataFrame(columns = fileNames, index = np.arange(len(subperiod))+1)

for i in range(len(subperiod)):
    df_select = df_mktcap[(df_mktcap.index >= subperiod[i][0]) & (df_mktcap.index <= subperiod[i][1])]
    length = len(df_select)
    for fileName in fileNames:
        non_NA = df_select[fileName].count()
        if non_NA*1.0/length >= 0.75:
            df_sampling[fileName].iloc[i] = 1

df_sampling

In [257]:
# for each time period, grab the stocks that are ready for sampling
sample_stocks_pool = []
all_mktcap_group = []

for i in range(len(subperiod)):
    # for each time period, grab the stocks that are ready for sampling
    sample_stocks = df_sampling.iloc[i].dropna().index.format()
    sample_stocks_pool.append(sample_stocks)
    
    # divide stocks in sampling pool into 5 mktcap group
    df_select = df_mktcap[(df_mktcap.index >= subperiod[i][0]) & (df_mktcap.index <= subperiod[i][1])]
    mktcap = df_select[sample_stocks]
    mean_mktcap = mktcap.apply(np.mean)
    bar = np.percentile(mean_mktcap,[20,40,60,80,100])
    mktcap_group = {}
    mktcap_group.update({'cap1': mean_mktcap[mean_mktcap<=bar[0]].index.format()})
    for j in range(4):
        group = mean_mktcap[(mean_mktcap<=bar[j+1]) & (mean_mktcap>bar[j])].index.format()
        key = 'cap'+str(j+2)
        mktcap_group.update({key:group})
        #ktcap_group.append(group) # from small cap to 
    all_mktcap_group.append(mktcap_group)

In [262]:
# number of stocks in each subperiod, each mktcap group
df_groupnumber = pd.DataFrame(index = np.arange(len(subperiod))+1, columns = ['cap1','cap2','cap3','cap4','cap5'])
for i in range(len(subperiod)):
    for j in range(5):
        key = 'cap'+str(j+1)
        df_groupnumber.iloc[i,j] = len(all_mktcap_group[i][key])
                       
df_groupnumber

,cap1,cap2,cap3,cap4,cap5
1,72,72,72,72,72
2,80,79,79,79,80
3,92,91,91,91,91
4,96,96,96,96,96
5,116,115,115,115,115
6,110,110,110,110,110
7,112,111,112,111,112


In [321]:
# calculate the volumn indicator for each stock in each subperiod
df_volumn = pd.DataFrame(columns=['ticker','period','mktcap','tao1','tao2','vol'])

for j in range(len(fileNames)):

    fileName = fileNames[j]

    for i in range(len(subperiod)):
        cap_cat = 'nan'
        # find its category in mktcap
        mktcap_group = all_mktcap_group[i]
        for (k,v) in  mktcap_group.items(): 
            if fileName in v:
                cap_cat = k

        # find its category in vol
        df = pd.read_csv(path+'\\'+fileName+'.csv', parse_dates=True, index_col=['date'],usecols=['SHROUT','VOL','date'])
        this_df = df[(df.index >= subperiod[i][0]) & (df.index <= subperiod[i][1])]
        half1 = this_df[:len(this_df)/2]
        half2 = this_df[len(this_df)/2:]
        tao1 = np.sum(half1['VOL'])/np.mean(half1['SHROUT'])
        tao2 = np.sum(half2['VOL'])/np.mean(half2['SHROUT'])
        vol_cat = 'normalVol'
        if tao1>1.2*tao2:
            vol_cat = 'decreacingVol'
        elif tao2>1.2*tao1:
            vol_cat = 'increasingVol'

        df_volume.loc[j + i*len(fileNames)] = [fileName,int(i+1),cap_cat,tao1,tao2,vol_cat]


df_volume

,ticker,period,mktcap,tao1,tao2,vol
0,AAL_ALEXANDER & ALEXANDER SVCS INC,1.0,cap3,1937.531517,2227.593889,normalVol
1,AAL_AMERICAN AIRLINES GROUP INC,1.0,nan,NaN,NaN,normalVol
2,AAPL_APPLE INC,1.0,nan,NaN,NaN,normalVol
3,AAP_ADVANCE AUTO PARTS INC,1.0,nan,NaN,NaN,normalVol
4,AAV_AVATEX CORP,1.0,nan,NaN,NaN,normalVol
5,AA_ALCOA CORP,1.0,nan,NaN,NaN,normalVol
6,AA_ALCOA INC,1.0,nan,NaN,NaN,normalVol
7,ABBV_ABBVIE INC,1.0,nan,NaN,NaN,normalVol
8,ABC_AMERICAN BROADCASTING COS INC,1.0,cap4,1899.914241,2863.273121,increasingVol
9,ABC_AMERISOURCEBERGEN CORP,1.0,nan,NaN,NaN,normalVol


In [322]:
df_volume.to_csv(path_or_buf = path+'\\'+'VolumeGroup.csv')

In [251]:
# sampling from each subperiod
import random
def sampleStocks(tickers,m):
    '''
    randomly sample m numbers from n numbers
    return the sampled tickers
    '''
    n = len(tickers)
    sampled = np.array(random.sample(range(n), m))
    sampled_tickers = []
    
    for i in sampled:
        sampled_tickers.append(tickers[i])
    return sampled_tickers
 
    
    
i = 0
mktcap_group = all_mktcap_group[0]
tickers = mktcap_group['cap1']
sampled_tickers = sampleStocks(tickers,20)

In [319]:
# total return and close price for pattern recognition

df_ret = pd.DataFrame()
df_prc = pd.DataFrame()

for fileName in fileNames:
    
    df = pd.read_csv(path+'\\'+fileName+'.csv', parse_dates=True, index_col=['date'])
    df_ret_new = pd.DataFrame(data=df['RET'])
    df_ret_new.columns = [fileName]
    df_prc_new = pd.DataFrame(data=df['PRC'])
    df_prc_new.columns = [fileName]
    df_ret = pd.concat([df_ret,df_ret_new],axis=1)
    df_prc = pd.concat([df_prc,df_prc_new],axis=1)

df_ret.to_csv(path_or_buf = path+'\\'+'Ret.csv')
df_prc.to_csv(path_or_buf = path+'\\'+'Prc.csv')

AAL_ALEXANDER & ALEXANDER SVCS INC
AAL_AMERICAN AIRLINES GROUP INC
AAPL_APPLE INC
AAP_ADVANCE AUTO PARTS INC
AAV_AVATEX CORP
AA_ALCOA CORP
AA_ALCOA INC
ABBV_ABBVIE INC
ABC_AMERICAN BROADCASTING COS INC
ABC_AMERISOURCEBERGEN CORP
ABI_APPLIED BIOSYSTEMS INC DEL
ABS_ALBERTSONS INC
ABT_ABBOTT LABORATORIES
ABX_BARRICK GOLD CORP
ACAS_AMERICAN CAPITAL LTD
ACF_A C F INDUSTRIES INC
ACK_ARMSTRONG HOLDINGS INC
ACN_ACCENTURE PLC IRELAND
ACS_AFFILIATED COMPUTER SERVICES INC
ACV_ALBERTO CULVER CO NEW
ACV_ALBERTO CULVER CO
ACY_AMERICAN CYANAMID CO
ADBE_ADOBE SYSTEMS INC
ADCT_A D C TELECOMMUNICATIONS INC
ADI_ANALOG DEVICES INC
ADM_ARCHER DANIELS MIDLAND CO
ADNT_ADIENT PLC
ADP_AUTOMATIC DATA PROCESSING INC
ADSK_AUTODESK INC
ADS_ALLIANCE DATA SYSTEMS CORP
ADT_A D T CORP
AD_AMSTED INDUSTRIES INC
AEE_AMEREN CORP
AEP_AMERICAN ELECTRIC POWER CO INC
AES_A E S CORP
AET_AETNA INC NEW
AET_AETNA INC
AFL_AFLAC INC
AFN_AMERIFIN CORP
AFS_ASSOCIATES FIRST CAPITAL CORP
AGC_AMERICAN GENERAL CORP
AGM_AMALGAMATED SUGAR 

DDS_DILLARDS INC
DD_DU PONT E I DE NEMOURS & CO
DEC_DIGITAL EQUIPMENT CORP
DEN_DENNYS INC
DE_DEERE & CO
DFS_DISCOVER FINANCIAL SERVICES
DF_DEAN FOODS CO NEW
DF_DEAN FOODS CO
DGN_DATA GENERAL CORP
DGX_QUEST DIAGNOSTICS INC
DG_ASSOCIATED DRY GOODS CORP
DG_DOLLAR GENERAL CORP NEW
DG_DOLLAR GENERAL CORP
DHI_D R HORTON INC
DHR_DANAHER CORP
DIGI_D S C COMMUNICATIONS CORP
DISCA_DISCOVERY COMMUNICATIONS INC
DIS_DISNEY WALT CO
DI_DRESSER INDUSTRIES INC
DJ_DOW JONES & CO INC
DLPH_DELPHI AUTOMOTIVE PLC
DLR_DIGITAL REALTY TRUST INC
DLTR_DOLLAR TREE INC
DLX_DELUXE CORP
DM_DOME MINES LTD
DNA_GENENTECH INC
DNB_DUN & BRADSTREET CORP DEL NEW
DNR_DENBURY RESOURCES INC
DN_DIAMOND INTERNATIONAL CORP
DOC_DOCTOR PEPPER CO
DOV_DOVER CORP
DO_DIAMOND OFFSHORE DRILLING INC
DPS_DR PEPPER SNAPPLE GROUP INC
DRI_DARDEN RESTAURANTS INC
DTE_D T E ENERGY CO
DTV_DIRECTV
DUK_DUKE ENERGY CORP NEW
DUK_DUKE ENERGY CORP
DVA_DAVITA INC
DVMT_DELL TECHNOLOGIES INC
DVN_DEVON ENERGY CORP NEW
DVN_DEVON ENERGY CORP
DWD_DEAN WITTER

KOE_KOEHRING COMPANY
KOP_KOPPERS CO
KORS_MICHAEL KORS HOLDINGS LIMITED
KO_COCA COLA CO
KRB_M B N A CORP
KRFT_KRAFT FOODS GROUP INC
KRI_KNIGHT RIDDER INC
KR_KROGER COMPANY
KSE_KEYSPAN ENERGY CORP
KSE_KEYSPAN ENERGY
KSS_KOHLS CORP
KSU_KANSAS CITY SOUTHERN
KWP_KING WORLD PRODUCTIONS INC
K_KELLOGG CO
LB_L BRANDS INC
LCE_LONE STAR INDUSTRIES INC
LDG_LONGS DRUG STORES INC
LDOS_LEIDOS HOLDINGS INC
LEG_LEGGETT & PLATT INC
LEH_LEHMAN BROTHERS HOLDINGS INC
LEN_LENNAR CORP
LGT_LIGGETT GROUP INC
LH_LABORATORY CORP AMERICA HLDGS
LIFE_LIFE TECHNOLOGIES CORP
LINB_LIN BROADCASTING CORP
LIT_LITTON INDUSTRIES INC
LI_LAIDLAW INTERNATIONAL INC
LKQ_L K Q CORP
LKS_LUCKY STORES INC
LLL_L 3 COMMUNICATIONS HLDGS INC
LLTC_LINEAR TECHNOLOGY CORP
LLX_LOUISIANA LAND & EXPLORATION CO
LLY_LILLY ELI & CO
LMT_LOCKHEED MARTIN CORP
LM_LEGG MASON INC
LNC_LINCOLN NATIONAL CORP
LNT_ALLIANT ENERGY CORP
LOR_LORAL CORP
LOTS_LOTUS DEVELOPMENT CORP
LOW_LOWES COMPANIES INC
LO_LORILLARD INC
LPX_LOUISIANA PACIFIC CORP
LRCX_LAM RES

ROH_ROHM & HAAS CO
ROK_ROCKWELL AUTOMATION INC
ROP_ROPER CORP
ROP_ROPER TECHNOLOGIES INC
ROST_ROSS STORES INC
RRC_RANGE RESOURCES CORP
RRD_DONNELLEY R R & SONS CO
RSG_REPUBLIC SERVICES INC
RS_REPUBLIC STEEL CORP
RTN_RAYTHEON CO
RVB_REVERE COPPER & BRASS INC
RVS_REEVES BROTHERS INC
RXR_REVCO D S INC NEW
RYAN_RYANS RESTAURANT GROUP INC
RYC_RAYCHEM CORP
RYI_RYERSON HOLDING CORP
RYN_RYAN HOMES INC
RYR_RYMER FOODS INC
R_RYDER SYSTEMS INC
R_UNIROYAL INC
SAA_SANDERS ASSOCIATES INC
SAF_SAFECO CORP
SAF_SANTA FE INTERNATIONAL CORP
SAI_SUNAMERICA INC
SANM_SANMINA CORP
SAPE_SAPIENT CORP
SBL_SYMBOL TECHNOLOGIES INC
SBUX_STARBUCKS CORP
SCG_SCANA CORP NEW
SCG_SCANA CORP
SCHW_SCHWAB CHARLES CORP NEW
SCI_SERVICE CORP INTERNATIONAL
SCI_STORER COMMUNICATIONS INC
SCV_S C A SERVICES INC
SDS_SUNGARD DATA SYSTEMS INC
SEBL_SIEBEL SYSTEMS INC
SED_SEDCO INC
SEE_SEALED AIR CORP NEW
SEE_SEALED AIR CORP
SEG_SEAGATE TECHNOLOGY INC
SE_7 ELEVEN INC
SE_SPECTRA ENERGY CORP
SFA_SCIENTIFIC ATLANTA INC
SFF_SANTA FE INDS I